In [3]:
# First importing required packages

import pandas as pd
import numpy as np
import os

In [4]:
path = os.path.join(os.getcwd(), "../data/2023data.csv")

# Ensure that this is the correct path
print(path)

df = pd.read_csv(path)


/Users/alexwang/Walkable/data_ops/../data/2023data.csv


/var/folders/w0/r7f6nr2x1xg6_lvr6slny5b40000gn/T/ipykernel_7303/2067589469.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [6]:
# Just useful for viewing the first couple entries
print(df.head())

  INCIDENT_NUMBER  OFFENSE_CODE  OFFENSE_CODE_GROUP  \
0       232007173          3115                 NaN   
1       232098649          3201                 NaN   
2       232098689          3831                 NaN   
3       232098474          3115                 NaN   
4       232098505           619                 NaN   

                     OFFENSE_DESCRIPTION DISTRICT REPORTING_AREA  SHOOTING  \
0                     INVESTIGATE PERSON       B3                        0   
1               PROPERTY - LOST/ MISSING       C6            207         0   
2  M/V - LEAVING SCENE - PROPERTY DAMAGE       B3            434         0   
3                     INVESTIGATE PERSON      C11            359         0   
4                     LARCENY ALL OTHERS       D4            167         0   

         OCCURRED_ON_DATE  YEAR  MONTH DAY_OF_WEEK  HOUR  UCR_PART  \
0  2023-01-27 22:44:00+00  2023      1   Friday       22       NaN   
1  2023-12-03 17:02:00+00  2023     12   Sunday       17    

In [55]:
print(df.columns)
df_clean = df[['INCIDENT_NUMBER', 'OFFENSE_CODE', 'OCCURRED_ON_DATE', 'Lat', 'Long', 'Location', 'STREET']]

print(df_clean.head())

Index(['INCIDENT_NUMBER', 'OFFENSE_CODE', 'OFFENSE_CODE_GROUP',
       'OFFENSE_DESCRIPTION', 'DISTRICT', 'REPORTING_AREA', 'SHOOTING',
       'OCCURRED_ON_DATE', 'YEAR', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'UCR_PART',
       'STREET', 'Lat', 'Long', 'Location'],
      dtype='object')
  INCIDENT_NUMBER  OFFENSE_CODE        OCCURRED_ON_DATE        Lat       Long  \
0       232007173          3115  2023-01-27 22:44:00+00  42.271661 -71.099535   
1       232098649          3201  2023-12-03 17:02:00+00  42.349088 -71.041135   
2       232098689          3831  2023-12-04 18:49:00+00  42.284659 -71.078401   
3       232098474          3115  2023-12-04 05:51:00+00  42.295248 -71.071088   
4       232098505           619  2023-12-02 14:15:00+00  42.339542 -71.069409   

                                   Location        STREET  
0  (42.271661031027065, -71.09953455161765)      FAVRE ST  
1  (42.349087870068175, -71.04113540160263)    SEAPORT LN  
2   (42.28465946403694, -71.07840062397692)  THETFO

In [56]:
LatMIN, LatMAX = df_clean['Lat'].min(), df_clean['Lat'].max()
LongMIN, LongMAX = df_clean['Long'].min(), df_clean['Long'].max()

print(LatMIN, LatMAX), print(LongMIN, LongMAX)

42.17145584778691 42.39504103740181
-71.20251098224917 -70.95372676830739


(None, None)

In [57]:
print(df_clean['OCCURRED_ON_DATE'].head())

0    2023-01-27 22:44:00+00
1    2023-12-03 17:02:00+00
2    2023-12-04 18:49:00+00
3    2023-12-04 05:51:00+00
4    2023-12-02 14:15:00+00
Name: OCCURRED_ON_DATE, dtype: object


In [58]:
df_clean = df_clean.copy()
df_clean['OCCURRED_ON_DATE'] = pd.to_datetime(df_clean['OCCURRED_ON_DATE'], utc=True)


In [59]:
df_clean.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OCCURRED_ON_DATE,Lat,Long,Location,STREET
0,232007173,3115,2023-01-27 22:44:00+00:00,42.271661,-71.099535,"(42.271661031027065, -71.09953455161765)",FAVRE ST
1,232098649,3201,2023-12-03 17:02:00+00:00,42.349088,-71.041135,"(42.349087870068175, -71.04113540160263)",SEAPORT LN
2,232098689,3831,2023-12-04 18:49:00+00:00,42.284659,-71.078401,"(42.28465946403694, -71.07840062397692)",THETFORD AVE
3,232098474,3115,2023-12-04 05:51:00+00:00,42.295248,-71.071088,"(42.29524811005633, -71.07108834551329)",REGINA RD
4,232098505,619,2023-12-02 14:15:00+00:00,42.339542,-71.069409,"(42.33954152609665, -71.06940928222296)",HARRISON AVE


In [63]:
# Now we need to rename the columns in order to have consistency between our PostgreSQL database and our .csv file that we want to generate

df_clean.rename(columns={'INCIDENT_NUMBER': 'incident_number', 'OFFENSE_CODE': 'offense_code', 'OCCURRED_ON_DATE': 'occurred_on_date', "Lat": "lat", "Long": "lon", "Location": 'location', "STREET": "street"}, inplace=True)

In [67]:
df_clean.head()

,incident_number,offense_code,occurred_on_date,lat,lon,location,street
0,232007173,3115,2023-01-27 22:44:00+00:00,42.271661,-71.099535,"(42.271661031027065, -71.09953455161765)",FAVRE ST
1,232098649,3201,2023-12-03 17:02:00+00:00,42.349088,-71.041135,"(42.349087870068175, -71.04113540160263)",SEAPORT LN
2,232098689,3831,2023-12-04 18:49:00+00:00,42.284659,-71.078401,"(42.28465946403694, -71.07840062397692)",THETFORD AVE
3,232098474,3115,2023-12-04 05:51:00+00:00,42.295248,-71.071088,"(42.29524811005633, -71.07108834551329)",REGINA RD
4,232098505,619,2023-12-02 14:15:00+00:00,42.339542,-71.069409,"(42.33954152609665, -71.06940928222296)",HARRISON AVE


In [65]:
print(len(df_clean))

131958


In [68]:
# Check for missing values
df_clean.isnull().sum()

# Filter rows where either 'lat' or 'lon' is missing
missing_values = df_clean[df_clean['lat'].isna() | df_clean['lon'].isna()]

# Show examples (first 5 rows with missing values)
print(missing_values.head())


    incident_number  offense_code          occurred_on_date  lat  lon  \
22        232070967          3114 2023-09-03 09:00:00+00:00  NaN  NaN   
48        242057267          2905 2024-07-12 11:20:00+00:00  NaN  NaN   
64        242057577          2647 2024-07-11 12:25:00+00:00  NaN  NaN   
82        232103621          1810 2023-12-20 00:00:00+00:00  NaN  NaN   
134       242007068          1106 2023-11-21 18:19:00+00:00  NaN  NaN   

    location          street  
22       NaN  E EAGLE STREET  
48       NaN    RIVER STREET  
64       NaN      SUDBURY ST  
82       NaN  OLDFIELDS ROAD  
134      NaN      SUDBURY ST  


## Important note about feature engineering/selection
Here I realized that there was quite a bit of missing longitude and latitude values. I decided to add a section specifically for streets as well. 

In [ ]:
# Further, because we are using a GIS system with points for the database, we will need to change the location to 'POINT' type
df_clean['location'] = df_clean.apply(lambda row: f"POINT({row['lon']} {row['lat']})", axis=1)


In [ ]:
# This creates the final DF that is cleaned and formatted the same as the desired table. 

df_clean.to_csv('/Users/alexwang/Walkable/data/cleaned_2023_data.csv', index=False)

In [1]:
import folium
from folium.plugins import HeatMap
import pandas as pd
import numpy as np

def create_heatmap(df, lat_column, lon_column, center_lat=None, center_lon=None, zoom_start=11):
    # If center coordinates are not provided, use the mean of the data
    if center_lat is None or center_lon is None:
        center_lat = df[lat_column].mean()
        center_lon = df[lon_column].mean()
    
    # Create a map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start)
    
    # Prepare the data for the heatmap
    heat_data = df[[lat_column, lon_column]].values.tolist()
    
    # Add the heatmap to the map
    HeatMap(heat_data).add_to(m)
    
    # Save the map
    m.save("heatmap.html")


In [9]:
    create_heatmap(df, 'Lat', 'Long')


ValueError: Location values cannot contain NaNs.